### Deep Learning Supervised Text Classification
- BiLSTM

Bidirectional recurrent neural networks (RNN) put two independent RNNs together. This structure allows the networks to have both backward and forward information about the sequence at every time step


### Prepare Environment

In [1]:
from numpy.random import seed
seed(156)
import tensorflow as tf
tf.random.set_seed(256)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab_Notebooks/safety_report_tc

/content/drive/MyDrive/Colab_Notebooks/safety_report_tc


In [4]:
# Importing required libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm.std import tqdm
from IPython.display import display, HTML
import time
import textwrap
import json
import datetime

### Load Data

In [5]:
# Load data from a minority category of interest
focus = 'hydraulic fluid or oil leak'
data_ref = '2209031206'
fn = f'data/prepared/{data_ref}_prepared_data.csv'
df = pd.read_csv(fn)
df

,text,category
0,foreign body entered employee l eye while grin...,0
1,drainage pipe damaged at twenty-two m depth se...,0
2,robodrill spider excavator being operated when...,1
3,pressure hose made contact with light fitting ...,0
4,nacap 30t hitachi hyd hose split resultong in ...,0
...,...,...
93852,fall on water vehicle an employee was using an...,0
93853,other fall to lower level unspecified an emplo...,0
93854,injured by slipping or swinging object held by...,0
93855,direct exposure to electricity greater than tw...,0


### Data Preparation (Train / Test Splits)

In [6]:
# Now create the input to the model training stage
X = df.text
y = df.category # Remember, these are not necessarily the 'truth' but rule_book hits

# Apply a simple 80/20 split
# TODO - The dataset is heavily imbalanced. Treat this appropriately.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [7]:
base_train_count = len(X_train)
base_train_count

75085

In [8]:
test_trues = list(y_test[y_test==1].index.values)
test_trues.sort()
print('Test:', len(test_trues), test_trues)

train_trues = list(y_train[y_train==1].index.values)
train_trues.sort()
print('Train:', len(train_trues), train_trues)

Test: 107 [17, 82, 343, 355, 514, 534, 1559, 1750, 1960, 1965, 2117, 2142, 2380, 2447, 2482, 2701, 2867, 3526, 3619, 3672, 3759, 3803, 3880, 3908, 3989, 4047, 4118, 4355, 4374, 4421, 4890, 5244, 5444, 6371, 6584, 6585, 6831, 6935, 7814, 7969, 8551, 8704, 8794, 9041, 9153, 9201, 9381, 9556, 9596, 9659, 9815, 9826, 9829, 11286, 11403, 11658, 11684, 11779, 12091, 12196, 12517, 12669, 12830, 12882, 13467, 13652, 13715, 15027, 16050, 18773, 20094, 20272, 20446, 21302, 21344, 21919, 22531, 22578, 22835, 22993, 23279, 23358, 24007, 24075, 24310, 26148, 26401, 26864, 29258, 30436, 30716, 35411, 50966, 51336, 55157, 57713, 61714, 64840, 69153, 70741, 76577, 77778, 78574, 79345, 81714, 82479, 88394]
Train: 470 [2, 24, 46, 143, 151, 235, 237, 241, 273, 404, 443, 454, 490, 518, 548, 751, 813, 897, 994, 1024, 1188, 1296, 1373, 1399, 1401, 1418, 1461, 1513, 1531, 1537, 1546, 1556, 1558, 1587, 1678, 1684, 1704, 1754, 1774, 1812, 1857, 1883, 2041, 2043, 2066, 2075, 2120, 2123, 2124, 2131, 2199, 2215, 

In [9]:
# Look at training 'trues', i.e., rule-book hits
df_train_trues = df.iloc[train_trues]
df_train_trues.to_csv('data/prepared/df_train_trues.csv')
df_train_trues

,text,category
2,robodrill spider excavator being operated when...,1
24,fuel red diesel spilled onto whiterockdirt sur...,1
46,oil leak from crane that was on site for plann...,1
143,small oil spill on twenty-one thousand and twe...,1
151,spider excavator hyd leak to soil 10ml spider ...,1
...,...,...
90259,struck by discharged object or substance an em...,1
90458,injured by slipping or swinging object held by...,1
90618,struck by swinging part of powered vehicle an ...,1
91361,caught in running equipment or machinery durin...,1


## Build Model

In [10]:
# Load modelling building libraries
from keras.layers import Dropout, Dense, Embedding, LSTM, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from sklearn import metrics
from sklearn.utils import shuffle
from collections import Counter
import numpy as np
import pickle
import matplotlib.pyplot as plt
import warnings
import logging
logging.basicConfig(level=logging.INFO)

In [11]:
# Create a function to prepare model input sequences and embedding dictionary
def prepare_model_input(X_train, X_test, MAX_NB_WORDS=75000, MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    all_text = np.concatenate((X_train, X_test), axis=0)
    all_text = np.array(all_text)

    # Fit tokeniser only on training text
    text = np.array(X_train)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)

    # Save the tokenizer as .pkl file
    pickle.dump(tokenizer, open(f'output/{time.strftime("%y%m%d%H%M")}_text_tokenizer.pkl', 'wb'))

    # Generate sequences for all text using tokenizer created only on training text
    # This converts the sentence into a sequence of integers, e.g., [2, 4, 5, 7]
    sequences = tokenizer.texts_to_sequences(all_text)

    # Get a list of all words and their sequence numbers
    word_index = tokenizer.word_index

    # Pad out the sequences with zeroes to max sequence length
    all_text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    
    # Seperate training sequences from test
    X_train_Glove = all_text[0:len(X_train), ]
    X_test_Glove = all_text[len(X_train):, ]

    # Now gather the embeddings
    # Start with standard GloVe
    ## https://www.google.com/search?client=safari&rls=en&q=glove+embeddings&ie=UTF-8&oe=UTF-8
    embeddings_dict = {}
    f = open("embeddings/glove/glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_dict[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_dict))
    return (X_train_Glove, X_test_Glove, word_index, embeddings_dict)


In [12]:
# Create a function that builds the deep learning model
def build_bilstm(word_index, embeddings_dict, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5, hidden_layer = 3, lstm_node = 32):
    # Initialize a sequential model
    model = Sequential()
    
    # Make the embedding matrix using the embedding_dict
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_dict.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
            
    # Add embedding layer
    print('_words', len(word_index))
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    # Add hidden layers 
    # Default will be 3 layers
    # Default lstm nodel number will be 32
    for i in range(0, hidden_layer):
        # Add a bidirectional lstm layer
        model.add(Bidirectional(LSTM(lstm_node, return_sequences=True, recurrent_dropout=0.2)))

        # Add a dropout layer after each lstm layer
        model.add(Dropout(dropout))

    model.add(Bidirectional(LSTM(lstm_node, recurrent_dropout=0.2)))

    # Attention addition
    #model.add(attention(return_sequences=True))
    
    model.add(Dropout(dropout))

    # Add the fully connected layer with 256 nuerons & Relu activation
    model.add(Dense(256, activation='relu'))

    # Add the output layer with softmax activation (binary output)
    model.add(Dense(nclasses, activation='softmax'))

    # Compile the model using sparse_categorical_crossentropy
    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=["categorical_accuracy"])
    return model

In [13]:
print("Preparing model input ...")
X_train_Glove, X_test_Glove, word_index, embeddings_dict = prepare_model_input(X_train, X_test)
print("Done!")

Preparing model input ...
Total 400000 word vectors.
Done!


In [26]:
print("Building Model!")
model = build_bilstm(word_index, embeddings_dict, 2)
model.summary()

Building Model!
_words 69944
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           3497250   
                                                                 
 bidirectional_4 (Bidirectio  (None, 500, 64)          21248     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 500, 64)           0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 500, 64)          24832     
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 500, 64)           0         
                                                                 
 bidirectional_6 (Bidirec

In [15]:
# Quick view of rule_book hits (should represent the focus category)
X_test[y_test == 1].iloc[0]

'a skid steer hydraulic hose broke causing no more than five gallons of hydraulic fluid to leak onto the ground at approximately one thousand, six hundred and fifteen on two million, one hundred and fifty-two thousand and twenty-two a logistics operator was transporting a skid steer to the entrance of block 3c for relocation a hydraulic hose on the boom of the skid steer broke causing less than five gallons of hydraulic fluid to leak onto the ground the leak was immediately noticed and the machine was stopped environmental swppp was notified and clean up procedures began the actual severity and potential ratings were selected as an a1 due to minimal reversible environmental impact the equipment was shut down environmental was notified clean up procedures immediately took place all contaminated soil and diapers were picked up and disposed of into proper containments'

### Model Training & Evaluatrion

In [16]:
def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    precision = (tp)/(tp+fp)
    recall = (tp)/(tp+fn)
    f1 = (2*(precision*recall))/(precision+recall)
    return {
        "mcc": mcc,
        "true positive": tp,
        "true negative": tn,
        "false positive": fp,
        "false negative": fn,
        "precision" : precision,
        "recall" : recall,
        "F1" : f1,
        "accuracy": (tp+tn)/(tp+tn+fp+fn)
    }

def compute_metrics(labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string], '')
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()


### **Basic Data Augmentation (BDA) Experiments**

In [17]:
# Load fabricated reports
fn_name = 'hydraulic fluid or oil leak_4700_simple_da'
df_fab = pd.read_csv(f'data/fabricated/{fn_name}.csv')
df_fab

,text,category
0,robodrill spider excavator being operated when...,1
1,robodrill spider excavator being operated when...,1
2,robodrill spider excavator being operated when...,1
3,robodrill spider excavator being operated hyd ...,1
4,robodrill excavator being when hose fitting fa...,1
...,...,...
4695,against object or equipment unspecified an emp...,1
4696,struck against object or equipment unspecified...,1
4697,struck against object or unspecified an employ...,1
4698,struck against object well kept or equipment u...,1


In [18]:
# Prepare data & conbine with original training data
X_train_fab = df_fab.text
X_train_new = X_train.append(X_train_fab)
X_train_list = list(X_train_new)
y_train_list = list(y_train)

In [19]:
# Complete the new training dataframe
df_tmp = pd.DataFrame(X_train_list, columns = ['text'])
y_train_list.extend([1 for i in range(len(X_train_fab))])
df_tmp['category'] = y_train_list
df_tmp

,text,category
0,ip operated air assisted door with left hand w...,0
1,twenty x spiders fell from roof when generator...,0
2,struck by discharged object or substance an em...,0
3,caught in running equipment or machinery nec a...,0
4,caught in running equipment or machinery durin...,0
...,...,...
79780,against object or equipment unspecified an emp...,1
79781,struck against object or equipment unspecified...,1
79782,struck against object or unspecified an employ...,1
79783,struck against object well kept or equipment u...,1


In [20]:
# Shuffle the dataframe
df_tmp = df_tmp.sample(frac=1).reset_index(drop=True)
df_tmp

,text,category
0,struck by falling object or equipment unspecif...,0
1,struck by falling object or equipment unspecif...,0
2,struck by swinging part of powered vehicle an ...,1
3,other fall to lower level unspecified an emplo...,0
4,wood team member tested positive for covid19 e...,0
...,...,...
79780,jackknifed or overturned roadway at approximat...,0
79781,fall on same level due to tripping over self a...,0
79782,direct exposure to electricity greater than tw...,0
79783,property damage to amdp06 cellar deck handrail...,0


In [21]:
# Visually check a random sample of 20 (hits)
df_tmp[df_tmp['category']==1].sample(20)

,text,category
55680,telehandler operating withing blackhillock sub...,1
28072,hydraulic oil leak scissor lift on the morning...,1
36941,fall on same level due to slipping an employee...,1
76022,earthquake remediation moxy suffered a hydraul...,1
36494,struck by discharged object chance on or subst...,1
5948,hydraulic oil release from main arm of the exc...,1
69374,a cat dozer hose failed spilling approximately...,1
13137,residual oil from tank was spilled onto ground...,1
323,telehandler operating withing blackhillock sub...,1
41926,a leak was found at crude oil storage tankful ...,1


In [22]:
X_train = df_tmp.text
y_train = df_tmp.category

In [23]:
print("Preparing model input ...")
X_train_Glove, X_test_Glove, word_index, embeddings_dict = prepare_model_input(X_train, X_test)
print("Done!")

Preparing model input ...
Total 400000 word vectors.
Done!


In [24]:
# Quick view of rule_book hits (should represent the focus category)
X_test[y_test == 1]

26401    a skid steer hydraulic hose broke causing no m...
12669    hydraulic oil from a burst excavator hose has ...
13652    hydraulic hose burst during quarry works a hyd...
514      faulty hydraulic hose hydraulic hose failed re...
82       minor spill to grade one liter from cat front ...
                               ...                        
51336    struck by discharged object or substance an em...
15027    release of preservation oil during inspection ...
4047     hb1117 hydraulic fitting leak on starboard cra...
355      hydraulic hose failure on a 360excavator resul...
76577    contact with hot objects or substances a linem...
Name: text, Length: 107, dtype: object

In [ ]:
# Train the model
history_fab_basic = model.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=4,
                              batch_size=128,
                              verbose=1)

Epoch 1/4
  4/624 [..............................] - ETA: 30:00 - loss: 0.5999 - categorical_accuracy: 0.8711

In [ ]:
# Print accuracy measures
print("\n Evaluating Model ... \n")

predicted = model.predict(X_test_Glove)
predicted = np.argmax(predicted, axis=1)

print(metrics.classification_report(y_test, predicted))
print("\n")
logger = logging.getLogger("logger")
result = compute_metrics(y_test, predicted)
for key in (result.keys()):
    logger.info("  %s = %s", key, str(result[key]))

In [ ]:
# Create dataframe for inspection of results
tmp = pd.DataFrame(X_test, columns=['text'])
tmp['pred'] = predicted
tmp['rule_book'] = y_test.values

# Quick look at predicted positives
print('Check rule-book hit count: ', len(tmp[tmp['rule_book']==1]))

In [ ]:
# Examine precision stats
# Look at predicted positives that did not have a rule-book hit
table = tmp[(tmp['pred']==1) & (tmp['rule_book']==0)]
display(HTML(table.to_html()))
print(f'\nThere are {len(table)} entries (out of {sum(y_test==1)})...')

In [ ]:
# Take a look at predicted negatives that had a rule-book hit
table = tmp[(tmp['pred']==0) & (tmp['rule_book']==1)]
display(HTML(table.to_html()))
print(f'\nThere are {len(table)} entries (out of {sum(y_test==1)})...')

In [ ]:
# Save the model
filename = f'output/models/{time.strftime("%y%m%d%H%M")}_{focus}_bilstm_bda_model.pkl'
pickle.dump(model, open(filename, 'wb'))